In [11]:
import numpy as np
from time import perf_counter
import cv2


In [12]:
x = np.ones((5, 7), dtype=np.uint8)


In [13]:

def integralImg(img):
    nr = img.shape[0]
    nc = img.shape[1]
    intImg = np.zeros((nr, nc))
    for r in range(nr):
        for c in range(nc):
            intImg[r, c] = img[r, c]
            if r > 0:
                intImg[r, c] += intImg[r-1, c]
            if c > 0:
                intImg[r, c] += intImg[r, c-1]
            if r > 0 and c > 0:
                intImg[r, c] -= intImg[r-1, c-1]
    return intImg


def integralImgLC(img):
    nr = img.shape[0]
    nc = img.shape[1]
    intImg = np.zeros((nr, nc))
    intImg[0, 0] = img[0, 0]
    intImg[0, 1:] = img[0, :-1].cumsum()
    intImg[1:, 0] = img[:-1, 0].cumsum(axis=0)
    intImg[1:, 1:] = (intImg[:-1, 1:] + intImg[1:, :-1] -
                      intImg[:-1, :-1]) + img[1:, 1:]
    return intImg


ticks = perf_counter()
intImg = integralImg(x)
ticks = perf_counter() - ticks
print("Time: ", ticks)
print(intImg)

# ticks = perf_counter()
# intImg = integralImgLC(x)
# ticks = perf_counter() - ticks
# print("Time2: ", ticks)
# print(intImg)


Time:  9.549999958835542e-05
[[ 1.  2.  3.  4.  5.  6.  7.]
 [ 2.  4.  6.  8. 10. 12. 14.]
 [ 3.  6.  9. 12. 15. 18. 21.]
 [ 4.  8. 12. 16. 20. 24. 28.]
 [ 5. 10. 15. 20. 25. 30. 35.]]


In [19]:
def SuavizadoIntegral(imgIntegral, r, c, n, m):
    N = 2*n+1
    M = 2*m+1
    nrows, ncols = imgIntegral.shape
    pad = (M - 1) // 2
    integralSuavizada = np.zeros((nrows, ncols))

    for r in range(nrows):
        for c in range(ncols):
            r0 = r - pad
            c0 = c - pad
            r1 = r + pad
            c1 = c + pad

            if r1 >= nrows:
                r1 = nrows - 1
            if c1 >= ncols:
                c1 = ncols - 1

            s0 = imgIntegral[r1, c1]
            s1 = 0
            s2 = 0
            s3 = 0

            if r0 > 0:
                s1 = imgIntegral[r0-1, c1]
            if c0 > 0:
                s2 = imgIntegral[r1, c0-1]
            if r0 > 0 and c0 > 0:
                s3 = imgIntegral[r0-1, c0-1]
            integralSuavizada[r, c] = s0 - s1 - s2 + s3
    return integralSuavizada


print(SuavizadoIntegral(intImg, 0, 0, 1, 1))


[[4. 6. 6. 6. 6. 6. 4.]
 [6. 9. 9. 9. 9. 9. 6.]
 [6. 9. 9. 9. 9. 9. 6.]
 [6. 9. 9. 9. 9. 9. 6.]
 [4. 6. 6. 6. 6. 6. 4.]]
